In [ ]:
!pip install numpy
#!pip install onnx onnxruntime pytest

In [ ]:
!pip install -e .

In [1]:
"""
Generating a ModelProto
=======================

This example demonstrates the use of *onnx-script* to define an ONNX model.
*onnx-script* behaves like a compiler. It converts a script into an ONNX model.

"""

#%%
# First, we define the implementation of a square-loss function in onnxscript.

import numpy as np
import onnx
from onnxruntime import InferenceSession

from onnxscript import FLOAT
from onnxscript import opset15 as op
from onnxscript import proto2text, script


@script()
def square_loss(X: FLOAT["N", 1], Y: FLOAT["N", 1]) -> FLOAT[1, 1]:  # noqa: F821
    diff = X - Y
    return op.ReduceSum(diff * diff, keepdims=1)


#%%
# We can convert it to a model (an ONNX *ModelProto*) as follows:

model = square_loss.to_model_proto()

#%%
# Let's see what the generated model looks like.
print(proto2text(model))

#%%
# We can run shape-inference and type-check the model using the standard ONNX API.

# model = onnx.shape_inference.infer_shapes(model)
# onnx.checker.check_model(model)

#%%
# And finally, we can use *onnxruntime* to compute the outputs
# based on this model, using the standard onnxruntime API.

sess = InferenceSession(model.SerializeToString())

X = np.array([[0, 1, 2]], dtype=np.float32).T
Y = np.array([[0.1, 1.2, 2.3]], dtype=np.float32).T

got = sess.run(None, {"X": X, "Y": Y})
expected = ((X - Y) ** 2).sum()

print(expected, got)

ImportError: cannot import name 'MapProto' from 'onnx.onnxproto' (/workspaces/onnx-script/onnx/onnxproto.py)

In [ ]:
!wget https://raw.githubusercontent.com/onnx/onnx/f1cd4167d5ffd6e0ffbb2e9d58e8e831aeebb428/onnx/onnx.in.proto
!sed 's/{PACKAGE_NAME}/onnx/g' onnx.in.proto

In [3]:
!wget https://raw.githubusercontent.com/onnx/onnx/f1cd4167d5ffd6e0ffbb2e9d58e8e831aeebb428/onnx/onnx-data.in.proto
!sed 's/{PACKAGE_NAME}/onnx/g' onnx-data.in.proto

// SPDX-License-Identifier: Apache-2.0


syntax = "proto2";

package onnx;
// #if ONNX-ML
import "onnx/onnx-ml.proto";
// #else
import "onnx/onnx.proto";
// #endif

// This file contains the proto definitions for MapProto and
// SequenceProto. These protos are used to represent the data structures
// of maps and sequence for use in test data or ModelProto.

// Sequences
//
// Defines a dense, ordered, collection of elements that are of homogeneous types.
// Sequences can be made out of tensors, maps, or sequences.
//
// If a sequence is made out of tensors, the tensors must have the same element
// type (i.e. int32). In some cases, the tensors in a sequence can have different
// shapes.  Whether the tensors can have different shapes or not depends on the
// type/shape associated with the corresponding "ValueInfo". For example,
// "Sequence<Tensor<float, [M,N]>" means that all tensors have same shape. However,
// "Sequence<Tensor<float, [omitted,omitted]>" means they can have different


In [ ]:
!pip install grpcio-tools # just an easy way to install protoc, which we need to compile the proto files into python data class thingies

In [11]:
!pip install "betterproto[compiler]"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 29.1 MB/s eta 0:00:00:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.6/96.6 kB 2.0 MB/s eta 0:00:00ta 0:00:01


In [15]:
!mkdir onnxproto
!python -m grpc_tools.protoc -I=. --python_betterproto_out=onnxproto onnx.in.proto

Writing __init__.py
Writing onnx.py


In [5]:
!mkdir onnxprotodata
!python -m grpc_tools.protoc -I=. --python_betterproto_out=onnxprotodata onnx-data.in.proto

mkdir: cannot create directory ‘onnxprotodata’: File exists
onnx/onnx-ml.proto: File not found.
onnx/onnx.proto: File not found.
onnx-data.in.proto:8:1: Import "onnx/onnx-ml.proto" was not found or had errors.
onnx-data.in.proto:10:1: Import "onnx/onnx.proto" was not found or had errors.
onnx-data.in.proto:53:12: "TensorProto" is not defined.
onnx-data.in.proto:57:12: "SparseTensorProto" is not defined.
onnx-data.in.proto:132:12: "TensorProto" is not defined.
onnx-data.in.proto:136:12: "SparseTensorProto" is not defined.
